In [0]:
%run "/Shared/_init_azure_conn"

In [0]:
# ============================================================
# Guided Capstone Step 4 – Analytical ETL (Updated Template)
# ============================================================
from datetime import datetime, timedelta

# Confirm ABFS access (optional quick check)
display(dbutils.fs.ls(f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/data/csv/"))

# (1) Spark Session and Imports
from pyspark.sql import SparkSession, functions as F, Window
from datetime import datetime, timedelta

spark = SparkSession.builder.appName("guided-step4-analytical-etl").getOrCreate()

# (2) Path Configuration
# ------------------------------------------------------------
# We reuse container_name and storage_account_name from _init_azure_conn
base_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net"
csv_path = f"{base_path}/data/csv/*/*/*.txt"
json_path = f"{base_path}/data/json/*/*/*.txt"
output_path = f"{base_path}/output_dir/"
eod_dir = f"{output_path}/eod"
analytical_dir = f"{output_path}/analytical"

print(f"✅ Base path: {base_path}")
print(f"📦 EOD path: {eod_dir}")
print(f"📊 Analytical path: {analytical_dir}")

# (3) Read Step 3 Outputs
trade_df = spark.read.parquet(f"{eod_dir}/trade")
quote_df = spark.read.parquet(f"{eod_dir}/quote")
trade_df.createOrReplaceTempView("trades")
quote_df.createOrReplaceTempView("quotes")

# (4) Moving Average Calculation
spark.sql("""
CREATE OR REPLACE TEMP VIEW tmp_trade_moving_avg AS
SELECT
    trade_dt,
    symbol,
    exchange,
    CAST(event_tm AS TIMESTAMP) AS event_tm_ts,
    event_seq_nb,
    trade_pr,
    AVG(trade_pr) OVER (
        PARTITION BY symbol
        ORDER BY CAST(event_tm AS TIMESTAMP)
        RANGE BETWEEN INTERVAL 30 MINUTES PRECEDING AND CURRENT ROW
    ) AS mov_avg_pr
FROM trades
""")

# Safely write to Delta with schema evolution
spark.table("tmp_trade_moving_avg") \
    .write \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable("temp_trade_moving_avg")

spark.table("tmp_trade_moving_avg").write.mode("overwrite").saveAsTable("temp_trade_moving_avg")

# (5) Previous Day Last Trade

# Base EOD path
trade_path = f"{eod_dir}/trade"

# Get all subdirectories under eod/trade (e.g., trade_dt=2020-07-29/)
dirs = [f.name for f in dbutils.fs.ls(trade_path) if f.name.startswith("trade_dt=")]

if not dirs:
    raise Exception(f"No trade_dt partitions found under {trade_path}")

# Extract trade_dt values from folder names
dates = [d.replace("trade_dt=", "").replace("/", "") for d in dirs]

# Pick the latest available date
latest_trade_dt = max(dates)
processing_date = datetime.strptime(latest_trade_dt, "%Y-%m-%d")

# Compute the prior date
prev_date = (processing_date - timedelta(days=1)).strftime("%Y-%m-%d")

print(f"✅ Latest trade_dt detected: {latest_trade_dt}")
print(f"📅 Previous trade_dt computed: {prev_date}")

spark.sql(f"""
CREATE OR REPLACE TEMP VIEW tmp_last_trade AS
SELECT
    symbol,
    exchange,
    MAX(event_tm) AS last_trade_tm,
    FIRST(trade_pr, TRUE) AS last_pr
FROM trades
WHERE trade_dt = '{prev_date}'
GROUP BY symbol, exchange
""")
spark.table("tmp_last_trade").write.mode("overwrite").saveAsTable("temp_last_trade")

# (6) Analytical Joins
spark.sql("""
CREATE OR REPLACE TEMP VIEW quote_union AS
SELECT
    trade_dt, symbol, exchange, event_tm, event_seq_nb,
    'Q' AS rec_type,
    bid_pr, bid_size, ask_pr, ask_size,
    NULL AS trade_pr, NULL AS mov_avg_pr
FROM quotes
UNION ALL
SELECT
    trade_dt, symbol, exchange, event_tm_ts AS event_tm, event_seq_nb,
    'T' AS rec_type,
    NULL AS bid_pr, NULL AS bid_size, NULL AS ask_pr, NULL AS ask_size,
    trade_pr, mov_avg_pr
FROM temp_trade_moving_avg
""")

spark.sql("""
CREATE OR REPLACE TEMP VIEW quote_union_update AS
SELECT *,
       LAST(trade_pr, TRUE) OVER (
           PARTITION BY symbol, exchange ORDER BY event_tm
           ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
       ) AS last_trade_pr,
       LAST(mov_avg_pr, TRUE) OVER (
           PARTITION BY symbol, exchange ORDER BY event_tm
           ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
       ) AS last_mov_avg_pr
FROM quote_union
""")

quote_update = spark.sql("""
SELECT trade_dt, symbol, exchange, event_tm, event_seq_nb,
       bid_pr, bid_size, ask_pr, ask_size,
       last_trade_pr, last_mov_avg_pr
FROM quote_union_update
WHERE rec_type = 'Q'
""")
quote_update.createOrReplaceTempView("quote_update")

# (7) Broadcast Join with Prior Day Close
quote_final = spark.sql("""
SELECT /*+ BROADCAST(temp_last_trade) */
    q.trade_dt, q.symbol, q.exchange, q.event_tm, q.event_seq_nb,
    q.bid_pr, q.ask_pr, q.bid_size, q.ask_size,
    q.last_trade_pr, q.last_mov_avg_pr,
    (q.bid_pr - t.last_pr) AS bid_pr_mv,
    (q.ask_pr - t.last_pr) AS ask_pr_mv
FROM quote_update q
LEFT JOIN temp_last_trade t
ON q.symbol = t.symbol AND q.exchange = t.exchange
""")

# *(8) Write Final Output
trade_date = processing_date.strftime("%Y-%m-%d")
analytical_out_path = f"{analytical_dir}/date={trade_date}"

quote_final.write.mode("overwrite").parquet(analytical_out_path)
print(f"✅ Analytical ETL results written to: {analytical_out_path}")

# (9) Optional Audit Summary
print("=== Analytical ETL Summary ===")
print(f"Trade Date:     {trade_date}")
print(f"Previous Date:  {prev_date}")
print(f"Output Path:    {analytical_out_path}")
print(f"Record Count:   {quote_final.count():,}")
print("🎯 Step 4 complete — ready for Step 5 orchestration.")
